In [ ]:
import numpy as np

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None


    def set_creator(self, func):
        self.creator = func


# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        fs = [self.creator]
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        while fs:
            f = fs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                fs.append(x.creator)



In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __call__(self, inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(xs)

        outputs = [Variable(as_array(y)) for y in ys]
        self.outputs = outputs
        for output in outputs:
            output.set_creator(self)

        return self.outputs

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [ ]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [ ]:
class Add(Function):
    def forward(self, xs):
        print("xs\t", xs)
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [ ]:
ab = [Variable(np.array(1)), Variable(np.array(2))]
add = Add()
y = add(ab)
y, y[0].data

In [ ]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

Give xs, a list as a parameter to Add(), 

then get the list type of output from it.

In [ ]:
xs = [Variable(np.array(0.6)), Variable(np.array(0.4))]
f = Add()
ys = f(xs)
    
y = ys[0]
y.data